In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses

import numpy as np
import cv2
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [3]:
import os, glob, re, u_net, dataset_configs, data_io

In [2]:
import sys, yaml
sys.path.append("/awlab/users/chsu/WorkSpace/tensorflow/segmentation/code")

In [5]:
file = "/awlab/users/chsu/WorkSpace/tensorflow/segmentation/code/configs/ic_nuc_paper.yaml"
with open(file) as f:
    cfg = yaml.safe_load(f)

cfg['val_cfg']


{'crop_size': None}

In [ ]:
task = 'incucyte_nucleus'
test_size = 0.2
random_state = 423

data_cfg = dataset_configs.get_dataset_config(task)
data_cfg['match_pattern'] = '_[A-Z]4_4_00d00h'

x_train_fnames, x_val_fnames, y_train_fnames, y_val_fnames = \
    data_io.get_data_filenames(**data_cfg, test_size=test_size, random_state=random_state)

num_train_data = len(x_train_fnames)
num_val_data = len(x_val_fnames)

for k in data_cfg:
    print(k, ':', data_cfg[k])
print()
print("Number of training samples: {}".format(num_train_data))
print("Number of validation samples: {}".format(num_val_data))

In [ ]:
for f in x_train_fnames + x_val_fnames:
    print(os.path.basename(f))

In [ ]:
1048/16

In [ ]:
file_dir = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/images'

filter_pattern = '_A(?!2)\d+_1_00d00h'
fnames = get_filenames(file_dir, '*.png', filter_pattern)
print(len(fnames))

for f in fnames:
    print(os.path.basename(f))


In [ ]:
img = cv2.imread(r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/p2017017086_ki67_merge/images/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67_G9_0000.png')
img.max()

In [ ]:
conv_1x1 = layers.Conv2D(3, (1, 1), activation='softmax')

x = tf.random.normal([1, 3, 3, 10])
y_pred = conv_1x1(x)
y_true = tf.random.uniform([1, 3, 3], 0, 3, dtype=tf.int32)
y_true = tf.one_hot(y_true, 3)

print(y_pred)
print(y_true)

In [ ]:
# loss = losses.sparse_categorical_crossentropy(y_true, y_pred)
loss = losses.categorical_crossentropy(y_true, y_pred)
print(loss)

In [ ]:
L = np.zeros_like(loss)

for i in range(L.shape[1]):
    for j in range(L.shape[2]):
#         c = tf.squeeze(y_true[0,i,j])
        c = y_true[0,i,j,:].numpy() == 1.
#         print(c)
        L[0,i,j] = -tf.math.log(y_pred.numpy()[0,i,j,c])
print(L)

In [ ]:
L[:,:,1:]